In [1]:
from gensim.models import Word2Vec
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
import os
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm

In [2]:
nltk.download('stopwords')

[nltk_data] Error loading stopwords: <urlopen error [WinError 10054]
[nltk_data]     Удаленный хост принудительно разорвал существующее
[nltk_data]     подключение>


False

In [3]:
def prepare_data(tokens):
    pattern = r'[^\w\s]'
    stop_words = set(stopwords.words('english'))
    
    cleaned_tokens = []
    for token in tokens:
        clean_token = re.sub(pattern, '', str(token).lower())
        if clean_token != '' and clean_token not in stop_words:
            cleaned_tokens.append(clean_token)
        
    return cleaned_tokens

In [4]:
def read_tsv_sentences(file_path: str) -> list[list[str]]:
    df = pd.read_csv(file_path, sep='\t', usecols=[0], header=None, names=['word'], skip_blank_lines=False, na_filter=False)
    words = df['word'].astype(str).str.strip().tolist()
    
    sentences = []
    current_sentence = []
    
    for word in words:
        if not word:  # Пустая строка в первом столбце
            if current_sentence:
                sentences.append(current_sentence)
                current_sentence = []
        else:
            current_sentence.append(word)
    
    # Добавляем последнее предложение, если оно есть
    if current_sentence:
        sentences.append(current_sentence)
    
    return sentences

In [6]:
train_data = []
test_data = []
base_path = '../assets/annotated-corpus/'

# Обработка данных
for data_type in tqdm(['train', 'test'], desc='Datasets'):
    data_path = os.path.join(base_path, data_type)
    
    if not os.path.exists(data_path):
        continue
        
    classes = ['0', '1', '2', '3']
    
    for class_name in tqdm(classes, desc=f'Classes ({data_type})'):
        class_path = os.path.join(data_path, class_name)
        
        docs = [
            f for f in os.listdir(class_path) 
            if f.endswith('.tsv') and os.path.isfile(os.path.join(class_path, f))
        ]
        
        for doc_name in docs:
            doc_path = os.path.join(class_path, doc_name)
            doc_id = os.path.splitext(doc_name)[0]  # Извлекаем doc_id без расширения
            sentences = read_tsv_sentences(doc_path)
            
            entry = {
                "doc_id": doc_id,
                "class": class_name,
                "words": sentences
            }
            
            if data_type == 'train':
                train_data.append(entry)
            else:
                test_data.append(entry)

Datasets: 100%|█████████████████████████████████████████████████████████████████████████| 2/2 [12:12<00:00, 366.08s/it]


In [19]:
model_path = "word2vec_v2.model"

In [25]:
def train_word2vec(train_data, model_path='word2vec.model'):    
    model = Word2Vec(
        vector_size=100,
        window=5,
        min_count=1,
        workers=4,
        sg=1,
        alpha=0.025,
        min_alpha=0.0001
    )
    
    print("\nПостроение словаря...")
    model.build_vocab(train_data)
    
    print("\nНачало обучения...")
    model.train(
        train_data,
        total_examples=model.corpus_count,
        epochs=10,
        compute_loss=True,
        report_delay=1
    )
    
    model.save(model_path)
    print(f"\nМодель сохранена в {model_path}")
    print(f"Размер словаря: {len(model.wv.key_to_index)}")
    print(f"Примеры слов: {list(model.wv.key_to_index.keys())[:10]}")
    
    return model

In [57]:
train_data[22]

{'doc_id': '100022',
 'class': '0',
 'words': [['Iraqi',
   'Prime',
   'Minister',
   'Encouraged',
   'to',
   'Meet',
   'Opponents',
   'An',
   'international',
   'conference',
   'on',
   'Iraq',
   'is',
   'expected',
   'to',
   'call',
   'on',
   'the',
   'government',
   'of',
   'Prime',
   'Minister',
   'Ayad',
   'Allawi',
   'to',
   'meet',
   'with',
   'its',
   'political',
   'opponents',
   'to',
   'encourage',
   'them',
   'to',
   'participate',
   'in',
   'the',
   "country's",
   'first',
   'democratic',
   'elections',
   'in',
   'January',
   ',',
   'according',
   'to',
   'a',
   'draft',
   'of',
   'the',
   "conference's",
   'final',
   'communique',
   '.']]}

In [26]:
train_data_sentences = [inner_list for item in train_data for inner_list in item['words']]

In [28]:
len(train_data_sentences)

170305

In [58]:
train_data[0]['

{'doc_id': '10000',
 'class': '0',
 'words': [['A',
   'Daily',
   'Look',
   'at',
   'U.S',
   '.',
   'Iraq',
   'Military',
   'Deaths',
   'AP',
   'AP',
   'As',
   'of',
   'Wednesday',
   ',',
   'Aug',
   '.'],
  ['25',
   ',',
   '964',
   'U.S',
   '.',
   'service',
   'members',
   'have',
   'died',
   'since',
   'the',
   'beginning',
   'of',
   'military',
   'operations',
   'in',
   'Iraq',
   'in',
   'March',
   '2003',
   ',',
   'according',
   'to',
   'the',
   'Defense',
   'Department',
   '.'],
  ['Of',
   'those',
   ',',
   '722',
   'died',
   'as',
   'a',
   'result',
   'of',
   'hostile',
   'action',
   'and',
   '242',
   'died',
   'of',
   'non',
   'hostile',
   'causes',
   '.']]}

In [30]:
model = train_word2vec(train_data_sentences, model_path)


Построение словаря...

Начало обучения...

Модель сохранена в word2vec_v2.model
Размер словаря: 94156
Примеры слов: ['.', 'the', ',', 'to', 'a', 'of', 'in', ';', 'and', 'on']


In [31]:
from typing import Union, List

def get_word_vectors(words, model, zero_vector_for_unknown=True):
    if isinstance(words, str):
        words = [words]
        
    vectors = []
    vector_size = model.vector_size if hasattr(model, 'vector_size') else 100
    
    for word in words:
        if word in model.wv:
            vectors.append(model.wv[word])
        else:
            if zero_vector_for_unknown:
                vectors.append(np.zeros(vector_size))
            else:
                continue  # Пропускаем отсутствующие слова
    
    return vectors

In [33]:
# Реализация косинусного расстояния
def cosine_distance(vec1: np.ndarray, vec2: np.ndarray) -> float:
    """Собственная реализация косинусного расстояния"""
    dot_product = np.dot(vec1, vec2)
    norm1 = np.linalg.norm(vec1)
    norm2 = np.linalg.norm(vec2)
    
    if norm1 == 0 or norm2 == 0:
        return 1.0  # Максимальное расстояние для нулевых векторов
    
    return 1 - (dot_product / (norm1 * norm2))

In [34]:
# Тестовые данные для анализа
test_words = {
    'press': {
        'similar': ['media', 'journalism', 'news'],
        'related': ['article', 'reporter', 'headline'],
        'different': ['mountain', 'guitar', 'painting']
    },
    'government': {
        'similar': ['administration', 'authority', 'regime'],
        'related': ['politics', 'law', 'policy'],
        'different': ['flower', 'bicycle', 'music']
    },
    'crime': {
        'similar': ['offense', 'violation', 'felony'],
        'related': ['police', 'law', 'punishment'],
        'different': ['sunshine', 'happiness', 'rainbow']
    },
    'troops': {
        'similar': ['soldiers', 'military', 'army'],
        'related': ['war', 'defense', 'combat'],
        'different': ['peace', 'butterfly', 'garden']
    }
}

In [35]:
# Функция для анализа расстояний
def analyze_distances(word, model, test_words):
    if word not in model.wv:
        raise ValueError(f"Слово '{word}' отсутствует в модели")
    
    main_vector = model.wv[word]
    distances = {'similar': [], 'related': [], 'different': []}
    
    for group, words in test_words[word].items():
        for w in words:
            if w in model.wv:
                dist = cosine_distance(main_vector, model.wv[w])
                distances[group].append(dist)
    
    return distances

In [36]:
metrics = {}
for word in test_words:
    distances = analyze_distances(word, model, test_words)
    metrics[word] = {
        'similar_mean': np.mean(distances['similar']),
        'related_mean': np.mean(distances['related']),
        'different_mean': np.mean(distances['different']),
        'diff_score': np.mean(distances['different']) - np.mean(distances['similar'])
    }

In [37]:
metrics

{'press': {'similar_mean': 0.5175909896691641,
  'related_mean': 0.595216711362203,
  'different_mean': 0.7673713614543279,
  'diff_score': 0.24978037178516388},
 'government': {'similar_mean': 0.4068453907966614,
  'related_mean': 0.4919247627258301,
  'different_mean': 0.7369668434063593,
  'diff_score': 0.33012145260969794},
 'crime': {'similar_mean': 0.577508936325709,
  'related_mean': 0.4980180859565735,
  'different_mean': 0.6868096192677816,
  'diff_score': 0.10930068294207262},
 'troops': {'similar_mean': 0.29604480663935345,
  'related_mean': 0.5652153690656027,
  'different_mean': 0.6560452431440353,
  'diff_score': 0.3600004365046819}}

In [43]:
# Из лабы 1
from nltk.stem.snowball import SnowballStemmer
from nltk.stem import WordNetLemmatizer

stemmer = SnowballStemmer(language='english')
lemmatizer = WordNetLemmatizer()

def split_into_sentences(text):
    # (?<!\w\.\w.) - проверяет, что нет слова, за которым следует точка и еще одно слово.
    # (?<![A-Z][a-z]\.) - проверяет, что перед текущей позицией нет заглавной буквы, за которой следует строчная буква и точка.
    # (?<=\.|\?|!) - проверяет, что перед текущей позицией находится точка, вопросительный знак или восклицательный знак.
    # \s - пробельный символ 
    sentence_pattern = re.compile(r'(?<!\w\.\w.)(?<![A-Z][a-z]\.)(?<=\.|\?|!)\s')
    
    # Находим все предложения в тексте
    sentences = sentence_pattern.split(text)
    
    # Удаляем пустые строки, если они есть
    sentences = [sentence.strip() for sentence in sentences if sentence.strip()]
    
    return sentences

def tokenize_sentence(sentence):
    # Регулярное выражение для выделения отдельных токенов
    pattern = r"\+?\b[\w@.]+(?:'\w+)?\b|[:;,?.!]"
    return re.findall(pattern, sentence)

def find_emails(sentence):
    # [A-Za-z0-9._%+-]+ - часть почты до @
    # [A-Za-z0-9.-]+ - доменная часть - .com(или другого)
    # [A-Z|a-z]{2,} - доменный уровень (например, .com, .org)
    email_pattern = re.compile(r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b')
    
    # Находим все email-адреса в тексте
    emails = email_pattern.findall(sentence)
    
    return emails

def find_phone_number(sentence):
    # \+? - символ + есть ноль или 1 раз
    # [- (]? и [- )]? - разделитель в  виде -, () и пробел
    # \d{3} - три любых цифры
    # 
    number_pattern = re.compile(r'\+?7?[- (]?\d{3}[- )]?\d{3}[- ]?\d{2}[- ]?\d{2}')
    
    # Находим все номера телефонов в тексте
    phone_numbers = number_pattern.findall(sentence)
    
    return phone_numbers

def find_dates(sentence):
    # Регулярное выражение для поиска дат
    date_pattern = re.compile(r'\b(\d{1,2})([./-]?)(\d{1,2})\2(\d{2,4})\b')
    
    # Находим все даты в тексте
    dates = date_pattern.findall(sentence)

    # Преобразуем найденные даты в строки
    formatted_dates = [f"{day}{separator}{month}{separator}{year}" for day, separator, month, year in dates]
    
    return formatted_dates

def tokenize_text(text):
    sentences = split_into_sentences(text)
    tokenized = []  # Список для хранения результатов по каждому предложению
    
    for sentence in sentences:
        sentence_data = {
            'tokens': [],  # Оригинальные токены
            'stems': [],   # Стемы
            'lemmas': [],  # Леммы
            'entities': []  # Специальные сущности (email, phone, date)
        }
        
        word_tokens = tokenize_sentence(sentence)
        
        for token in word_tokens:
            # Поиск специальных сущностей
            emails = find_emails(token)
            if emails:
                sentence_data['entities'].append({'type': 'email', 'value': emails[0]})
                continue
            
            phones = find_phone_number(token)
            if phones:
                sentence_data['entities'].append({'type': 'phone', 'value': phones[0]})
                continue
            
            dates = find_dates(token)
            if dates:
                sentence_data['entities'].append({'type': 'date', 'value': dates[0]})
                continue
            
            # Обычная обработка токена
            stem = stemmer.stem(token)
            lemma = lemmatizer.lemmatize(token)
            
            sentence_data['tokens'].append(token)
            sentence_data['stems'].append(stem)
            sentence_data['lemmas'].append(lemma)
        
        tokenized.append(sentence_data)  # Добавляем результат предложения в общий список
    
    return tokenized

In [44]:
text = 'This is an example of text. Second sentence.'

In [45]:
tokenize_text(text)

[{'tokens': ['This', 'is', 'an', 'example', 'of', 'text', '.'],
  'stems': ['this', 'is', 'an', 'exampl', 'of', 'text', '.'],
  'lemmas': ['This', 'is', 'an', 'example', 'of', 'text', '.'],
  'entities': []},
 {'tokens': ['Second', 'sentence', '.'],
  'stems': ['second', 'sentenc', '.'],
  'lemmas': ['Second', 'sentence', '.'],
  'entities': []}]

In [59]:
def vectorize_text(text, model, tokenize=True):
    if tokenize:
        tokenized_sentences = tokenize_text(text)
    else:
        tokenized_sentences = text
    vector_size = model.vector_size if hasattr(model, 'vector_size') else 100
    
    doc_token_vectors = []
    doc_sentence_vectors = []
    doc_token_level_vectors = []
    
    for sentence_data in tokenized_sentences:
        # Векторизация токенов
        tokens = sentence_data['tokens'] if tokenize else sentence_data
        token_vectors = get_word_vectors(tokens, model)
        
        # Сохраняем вектора токенов для текущего предложения
        doc_token_level_vectors.append(token_vectors)
        
        # Рассчитываем средний вектор предложения
        if len(token_vectors) > 0:
            sentence_vector = np.mean(token_vectors, axis=0)
        else:
            sentence_vector = np.zeros(vector_size)
        
        doc_sentence_vectors.append(sentence_vector)
        doc_token_vectors.extend(token_vectors)  # Все токены документа
    
    # Рассчитываем средние векторы
    result = {
        'sentence_vectors': doc_sentence_vectors,
        'token_vectors': doc_token_level_vectors
    }
    
    # Вектор всего документа (среднее по предложениям)
    if len(doc_sentence_vectors) > 0:
        result['document_vector'] = np.mean(doc_sentence_vectors, axis=0)
    else:
        result['document_vector'] = np.zeros(vector_size)
    
    # Дополнительно: среднее по всем токенам документа (альтернативный подход)
    if len(doc_token_vectors) > 0:
        result['document_vector_alt'] = np.mean(doc_token_vectors, axis=0)
    else:
        result['document_vector_alt'] = np.zeros(vector_size)
    
    return result

In [68]:
vectorized_text = vectorize_text(train_data[0]['words'], model, tokenize=False)

In [78]:
type(train_data[0])

dict

In [82]:
def vectorize_and_save_dataset(model, train_data, test_data, output_dir="../assets/annotated-corpus-vectorized/"):
    def _process_subset(data, subset_name):
        with tqdm(total=len(data), desc=f"Processing {subset_name} subset") as pbar:
            for doc in data:
                # Векторизация документа
                vectorization_result = vectorize_text(doc["words"], model, tokenize=False)
                doc_vector = vectorization_result["document_vector"]
                
                class_dir = os.path.join(output_dir, subset_name, doc["class"])
                os.makedirs(class_dir, exist_ok=True)
                
                # Сохранение в файл
                filename = os.path.join(class_dir, f"{doc['doc_id']}.tsv")
                with open(filename, "w", encoding="utf-8") as f:
                    vector_str = "\t".join(map("{:.6f}".format, doc_vector))
                    f.write(f"{doc['doc_id']}\t{vector_str}\n")
                
                pbar.update(1)
                pbar.set_postfix_str(f"Last processed: {doc['doc_id']}")

    # Обработка обучающих и тестовых данных
    _process_subset(train_data, "train")
    _process_subset(test_data, "test")

In [83]:
vectorize_and_save_dataset(model, train_data, test_data)

Processing test subset: 100%|█████████████████████████████████| 7600/7600 [01:36<00:00, 78.87it/s, Last processed: 998]
